# Depuración y creación de perfiles
En la vida de un programador profesional, la depuración y la resolución de problemas ocupan una cantidad significativa de tiempo. Aunque se trabaje en el código base más bello jamás escrito por un ser humano, seguirá habiendo errores; eso está garantizado.

Ser capaz de depurar código de forma rápida y eficaz es una habilidad que todo programador debe mejorar constantemente. Al igual que las pruebas, la depuración es una habilidad que se aprende mejor con la experiencia. Hay pautas que puedes seguir, pero no hay un libro mágico que te enseñe todo lo que necesitas saber para ser bueno en esto.

Algunos errores son muy fáciles de detectar. Surgen de errores groseros y, una vez que ves los efectos de esos errores, es fácil encontrar una solución que arregle el problema. Pero hay otros bugs que son mucho más sutiles, mucho más escurridizos, y requieren verdadera pericia y una gran dosis de creatividad y pensamiento fuera de lo común para ser tratados.

Los peores de todos son los no deterministas (opinión propia). A veces ocurren y a veces no. Algunas sólo ocurren en el entorno A, pero no en el B, aunque se suponga que A y B son exactamente iguales. Esos bugs son los verdaderamente malignos, y pueden volverte loco.

En esta sesión, intentaremos mostrarte algunas técnicas útiles que puedes emplear en función de la gravedad del fallo, y algunas sugerencias que, con suerte, potenciarán tus armas contra fallos y problemas.

En concreto, vamos a ver lo siguiente:
- [Técnicas de depuración]()
- [Pautas para la resolución de problemas]()
- [Creación de perfiles]()

# Técnicas de depuración
En esta parte, le presentaremos algunas de las técnicas que se utilizan a menudo. No es una lista exhaustiva, pero debería darte algunas ideas útiles para saber por dónde empezar cuando depures tu propio código Python.

## Depuración con print
La clave para entender cualquier fallo es comprender qué está haciendo tu código en el punto en el que se produce el fallo. Por esta razón, veremos algunas técnicas diferentes para inspeccionar el estado de un programa mientras se está ejecutando.

Probablemente la técnica más fácil de todas es añadir llamadas `print()` en varios puntos de tu código. Esto le permite ver fácilmente qué partes de su código se ejecutan, y cuáles son los valores de las variables clave en diferentes puntos durante la ejecución. Por ejemplo, si estás desarrollando un sitio web Django y lo que ocurre en una página no es lo que esperabas, puedes llenar la vista con impresiones y echar un ojo a la consola mientras recargas la página.

Hay varios inconvenientes y limitaciones en el uso de `print()` para la depuración. Para usar esta técnica, necesitas poder modificar el código fuente y ejecutarlo en un terminal donde puedas ver la salida de tus llamadas a la función `print()`. Esto no es un problema en su entorno de desarrollo en su propia máquina, pero limita la utilidad de esta técnica en otros entornos.

Cuando dispersa las llamadas `print()` en tu código, puede fácilmente terminar duplicando mucho código de depuración. Por ejemplo, puede que quieras imprimir marcas de tiempo, o de alguna manera construir una cadena con la información que quieres mostrar. Otro problema es que es extremadamente fácil olvidar las llamadas a print() en tu código.

Por estas razones, a veces es preferible usar una función de depuración personalizada en lugar de hacer llamadas a print().

## Depuración con una función personalizada
Tener una función de depuración personalizada en un fragmento que puedas coger y pegar rápidamente en el código puede ser muy útil. Si eres rápido, siempre puedes codificar una sobre la marcha. Lo importante es codificarla de forma que no deje cosas por ahí cuando finalmente elimines las llamadas y su definición. Por lo tanto, es importante codificarlo de manera que sea completamente autocontenido. Otra buena razón para este requisito es que evitará posibles conflictos de nombres con el resto del código.

Veamos un ejemplo de una función de este tipo:

In [2]:
def debug(*msg, print_separator=True):
    print(*msg)
    if print_separator:
        print('-' * 40)
debug('La data es ...')
debug('Diferentes', 'Strings', 'no son un problema')
debug('Después del bucle while', print_separator=False)

La data es ...
----------------------------------------
Diferentes Strings no son un problema
----------------------------------------
Después del bucle while


En este caso, estamos utilizando un argumento de sólo palabra clave para poder imprimir un separador, que es una línea de 40 guiones.

La función es muy simple. Simplemente redirigimos lo que haya en `msg` a una llamada a `print()` y, si `print_separator` es `True`, imprimimos un separador de líneas.

Esta es sólo una manera fácil de aumentar una simple llamada a la función print(). Veamos cómo podemos calcular una diferencia de tiempo entre llamadas, utilizando una de las características complicadas de Python a nuestro favor:

In [3]:
from time import sleep

def debug(*msg, timestamp=[None]):
    print(*msg)
    from time import time # local import
    if timestamp[0] is None:
        timestamp[0] = time() #1
    else:
        now = time()
        print(
            ' Time elapsed: {:.3f}s'.format(now - timestamp[0])
        )
        timestamp[0] = now #2

debug('Ingresando un código desagradable ...')
sleep(.3)
debug('Primer paso hecho.')
sleep(.5)
debug('Segundo paso hecho.')

Ingresando un código desagradable ...
Primer paso hecho.
 Time elapsed: 0.305s
Segundo paso hecho.
 Time elapsed: 0.500s


Esto es un poco más complicado, pero sigue siendo bastante sencillo. En primer lugar, observe que hemos utilizado una sentencia `import` dentro de nuestra función `debug()` para importar la función `time()` del módulo `time`. Esto nos permite evitar tener que añadir esa importación fuera de la función, y quizás olvidarla allí.

Observa cómo definimos `timestamp`. Es un parámetro de función con una lista como valor por defecto. En la Clase_03, Funciones, los bloques de construcción del código, advertimos contra el uso de valores por defecto mutables para los parámetros, porque el valor por defecto se inicializa cuando Python analiza la función y el mismo objeto persiste a través de diferentes llamadas a la función. La mayoría de las veces, este no es el comportamiento deseado. En este caso, sin embargo, estamos aprovechando esta característica para almacenar una marca de tiempo de la llamada anterior a la función, sin tener que utilizar una variable global externa.

Después de imprimir cualquier mensaje que tuviéramos que imprimir e importar `time()`, inspeccionamos el contenido del único elemento en `timestamp`. Si es `None`, no tenemos ninguna marca de tiempo anterior, así que establecemos el valor a la hora actual (#1). Por otro lado, si tenemos un `timestamp` anterior, podemos calcular una diferencia (que formateamos ordenadamente a tres dígitos decimales) y finalmente, ponemos la hora actual en `timestamp` (#2).

El uso de una función de depuración personalizada resuelve algunos de los problemas asociados al uso de `print()`. Reduce la duplicación de código de depuración y facilita la eliminación de todo el código de depuración cuando ya no lo necesite. Sin embargo, aún requiere modificar el código y ejecutarlo en una consola donde puedas inspeccionar la salida.

## Uso del depurador de Python
Otra forma muy efectiva de depurar en Python es utilizar un depurador interactivo. El módulo `pdb` de la biblioteca estándar de Python proporciona un depurador de este tipo; sin embargo, normalmente preferimos usar el paquete de terceros `pdbpp`. Este es un sustituto directo de `pdb`, con una interfaz de usuario algo más amigable y algunas herramientas adicionales útiles, una de ellas es el modo pegajoso (sticky mode), que te permite ver una función completa mientras recorres sus instrucciones.

Hay varias formas de activar el depurador (los mismos métodos sirven tanto para `pdb` como para `pdbpp`). El método más común es añadir una llamada invocando al depurador a tu código. Esto se conoce como añadir un punto de interrupción al código. Cuando el código se ejecuta y el intérprete alcanza el punto de interrupción, la ejecución se suspende y se obtiene acceso por consola a una sesión de depuración interactiva que permite inspeccionar todos los nombres del ámbito actual y recorrer el programa línea a línea. También puede modificar los datos sobre la marcha para cambiar el flujo del programa.

Como ejemplo de prueba, supongamos que tenemos un analizador sintáctico que está lanzando `KeyError` porque falta una clave en un diccionario. El diccionario es de una carga JSON que no podemos controlar, y sólo queremos, por el momento, hacer trampa y pasar ese control, ya que estamos interesados en lo que viene después. Veamos cómo podríamos interceptar este momento, inspeccionar los datos, arreglarlo, y llegar al fondo del asunto, con el depurador:

In [1]:
# d proviene de una carga JSON que no controlamos
d = {'first': 'v1', 'second': 'v2', 'fourth': 'v4'}
# keys también proviene de una carga JSON que no controlamos
keys = ('first', 'second', 'third', 'fourth')
def do_something_with_value(value):
    print(value)

for key in keys:
    do_something_with_value(d[key])

print('Validation done.')


v1
v2


KeyError: 'third'

La forma más común de hacerlo es importar `pdb` y llamas a su método `set_trace()`. Muchos desarrolladores tienen macros en su editor para añadir esta línea con un atajo de teclado. A partir de Python 3.7, sin embargo, podemos simplificar las cosas aún más usando `breakpoint()`:

In [6]:
# d proviene de una carga JSON que no controlamos
d = {'first': 'v1', 'second': 'v2', 'fourth': 'v4'}
# keys también proviene de una carga JSON que no controlamos
keys = ('first', 'second', 'third', 'fourth')

def do_something_with_value(value):
    print(value)

for key in keys:
    breakpoint()  # Este es el punto donde se detendrá la ejecución
    if key in d:
        do_something_with_value(d[key])
    else:
        print(f"Key '{key}' not found in dictionary 'd'.")

print('Validation done.')


v1
v2
Key 'third' not found in dictionary 'd'.
v4
Validation done.


La nueva función incorporada `breakpoint()` llama a `sys.breakpointhook()` bajo el capó, que está programado por defecto para llamar a `pdb.set_trace()`. Sin embargo, puedes reprogramar `sys.breakpointhook()` para llamar a lo que quieras, y por lo tanto `breakpoint()` apuntará a eso también, lo cual es muy conveniente.

En primer lugar, ten en cuenta que cuando llegas a un `breakpoint`, aparece una consola que te indica dónde estás (el módulo Python) y qué línea es la siguiente en ejecutarse. En este punto, puedes realizar un montón de acciones exploratorias, como inspeccionar el código antes y después de la siguiente línea, imprimir una traza de la pila e interactuar con los objetos. En nuestro caso, primero inspeccionamos la tupla keys. También inspeccionamos las claves de d. Vemos que falta 'tercera', así que la ponemos nosotros (¿podría ser peligroso? Piénsalo). Finalmente, ahora que todas las claves están dentro, escribimos c, que significa (c)ontinuar.

El depurador también te da la posibilidad de avanzar con tu código línea a línea usando (n)ext, para (s)tep en una función para un análisis más profundo, o para manejar interrupciones con (b)reak. Para obtener una lista completa de comandos, consulta la documentación (que puedes encontrar en https://docs.python.org/3.7/library/pdb.html) o escribe (h)elp en la consola del depurador.

Puedes ver, en la salida de la ejecución anterior, que finalmente pudimos llegar al final de la validación.

pdb (o pdbpp) es una herramienta inestimable que utilizamos todos los días. Así que ve y diviértete, establece un punto de interrupción en algún lugar e intenta inspeccionarlo, sigue la documentación oficial y prueba los comandos en tu código para ver su efecto y aprenderlos bien.

## Inspección de registros
Otra forma de depurar una aplicación que se comporta mal es inspeccionar sus registros. Un registro es una lista ordenada de eventos ocurridos o acciones realizadas durante la ejecución de una aplicación. Si un registro se escribe en un archivo en el disco, se conoce como archivo de registro.

El uso de registros para depuración es en cierto modo similar a añadir llamadas a `print()` o usar una función de depuración personalizada. La diferencia clave es que normalmente añadimos el registro a nuestro código desde el principio, para ayudar a la futura depuración, en lugar de añadirlo durante la depuración y luego eliminarlo de nuevo. Otra diferencia es que el registro puede configurarse fácilmente para que se envíe a un archivo o a una ubicación de red. Estos dos aspectos hacen que el registro sea ideal para depurar código que se está ejecutando en una máquina remota a la que puede que no tengas acceso directo.

El hecho de que el registro se añada normalmente al código antes de que se produzca un error plantea el reto de decidir qué registrar. Normalmente se espera encontrar entradas en los registros correspondientes al inicio y finalización (y potencialmente también pasos intermedios) de cualquier proceso importante que tenga lugar dentro de la aplicación. Los valores de las variables importantes deberían incluirse en estas entradas de registro. Los errores también deben ser registrados, de modo que si se produce un problema, podemos inspeccionar los registros para averiguar lo que salió mal.

Casi todos los aspectos del registro en Python pueden ser configurados de diferentes maneras. Esto nos da mucho poder, ya que podemos cambiar a dónde se envían los registros, qué mensajes de registro se envían, y cómo se formatean los mensajes de registro, simplemente reconfigurando el registro y sin cambiar ningún otro código. Los cuatro tipos principales de objetos involucrados en el registro en Python son:
- **Loggers:** Exponen la interfaz que el código de la aplicación utiliza directamente
- **Manejadores:** Envían los registros de log (creados por los loggers) al destino apropiado
- **Filtros:** Proporcionan un mecanismo más preciso para determinar los registros que se van a enviar.
- **Formateadores:** Especifican la disposición de los registros en la salida final.

El registro se realiza llamando a métodos de instancias de la clase `Logger`. Cada línea de registro tiene asociado un nivel de gravedad. Los niveles más utilizados son `DEBUG`, `INFO`, `WARNING`, `ERROR` y `CRITICAL`. Los registradores utilizan estos niveles para determinar qué mensajes de registro deben emitirse. Todo lo que esté por debajo del nivel del registrador será ignorado. Esto significa que debes tener cuidado de registrar en el nivel apropiado. Si registras todo en el nivel `DEBUG`, necesitarás configurar tu logger en (o por debajo de) el nivel `DEBUG` para ver cualquiera de tus mensajes. Esto puede dar lugar rápidamente a que tus archivos de registro se vuelvan extremadamente grandes. Un problema similar ocurre si registras todo en el nivel `CRITICAL`.

Python te da varias opciones de dónde registrar. Puedes registrar un archivo, una ubicación de red, una cola, una consola, las facilidades de registro de tu sistema operativo, y así sucesivamente. El lugar al que envíes tus registros dependerá en gran medida del contexto. Por ejemplo, cuando ejecutas tu código en tu entorno de desarrollo, normalmente lo registrarás en tu terminal. Si tu aplicación se ejecuta en una única máquina, puede que registres en un archivo o que envíes tus registros a las instalaciones de registro del sistema operativo. Por otro lado, si tu aplicación utiliza una arquitectura distribuida que se extiende por varias máquinas (como en el caso de las arquitecturas orientadas a servicios o microservicios), es muy útil implementar una solución centralizada para el registro, de modo que todos los mensajes de registro procedentes de cada servicio puedan almacenarse e investigarse en un único lugar. Es de gran ayuda, ya que, de lo contrario, intentar correlacionar archivos gigantescos de varias fuentes diferentes para averiguar qué ha ido mal puede convertirse en un verdadero desafío.

Tener en cuenta que una arquitectura orientada a servicios (SOA por sus siglás en inglés) es un patrón arquitectónico de diseño de software en el que los componentes de una aplicación prestan servicios a otros componentes a través de un protocolo de comunicaciones, normalmente en red. Lo bueno de este sistema es que, cuando se codifica adecuadamente, cada servicio puede escribirse en el lenguaje más apropiado para servir a su propósito. Lo único que importa es la comunicación con los demás servicios, que debe realizarse a través de un formato común para poder intercambiar datos.

Las arquitecturas de microservicios son una evolución de las SOA, pero siguen un conjunto diferente de patrones arquitectónicos.

El inconveniente de la configurabilidad del registro de Python es que la maquinaria de registro es algo compleja. La buena noticia es que a menudo no necesitas configurar mucho. Si empiezas de forma sencilla, en realidad no es tan difícil. Para probarlo, te mostraremos un ejemplo muy simple de registro de unos pocos mensajes a un archivo:

In [7]:
import logging

logging.basicConfig(
    filename='Clase_10.log',
    level=logging.DEBUG,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p')

mylist = [1, 2, 3]
logging.info("Starting to process 'mylist'...")

for position in range(4):
    try:
        logging.debug(
            'Value at position %s is %s', position, mylist[position]
        )
    except IndexError:
        logging.exception('Faulty position: %s', position)

logging.info("Done processing 'mylist'.")

Vamos a repasarlo línea por línea. Primero, importamos el módulo de `logging`, luego establecemos una configuración básica. Especificamos un nombre de archivo, configuramos el registrador para que emita cualquier mensaje de registro con nivel `DEBUG` o superior, y establecemos el formato del mensaje. Registraremos la información de fecha y hora, el nivel y el mensaje.

Con la configuración en su lugar, podemos empezar a registrar. Comenzamos registrando un mensaje de información que nos dice que estamos a punto de procesar nuestra lista. Dentro del bucle, registraremos el valor en cada posición (usamos la función `debug()` para registrar en el nivel `DEBUG`). Usamos `debug()` aquí para poder filtrar estos registros en el futuro (estableciendo el nivel mínimo a `logging.INFO` o más), porque podríamos tener que manejar listas muy grandes y no queremos registrar siempre todos los valores.

Si obtenemos `IndexError` (y lo obtenemos, ya que estamos haciendo un bucle sobre `range(4)`), llamamos a `logging.exception()`, que registra en el nivel `ERROR`, pero también muestra la traza de la `excepción`.

Al final del código, registramos otro mensaje de información para decir que hemos terminado. Después de ejecutar este código, tendremos un nuevo archivo `Clase_10.log`.

Esto es exactamente lo que necesitamos para poder depurar una aplicación que se está ejecutando en una máquina remota, en lugar de nuestro propio entorno de desarrollo. Podemos ver lo que pasó, el rastreo de cualquier excepción planteada, y así sucesivamente.

Logging es un arte. Necesitas encontrar un buen equilibrio entre registrar todo y no registrar nada. Lo ideal es que registres todo lo que necesites para asegurarte de que tu aplicación funciona correctamente, y posiblemente todos los errores o excepciones.

## Otrás técnicas
Terminaremos este apartado sobre depuración mencionando brevemente un par de técnicas más que pueden resultarte útiles.

### Lectura de rastreos
Los errores se manifiestan a menudo como excepciones no controladas. La capacidad de interpretar el rastreo de una excepción es, por tanto, una habilidad crucial para depurar con éxito. Es conveniente repasar el temea de Excepciones y Gestores de Contexto. Si estás tratando de entender por qué ocurrió una excepción, a menudo es útil inspeccionar el estado de tu programa en las líneas de rastreo que salen cuando lo ejecutas.

### Aserciones
Los errores son a menudo el resultado de suposiciones incorrectas en nuestro código. Las aserciones pueden ser muy útiles para validar esas suposiciones. Si nuestras suposiciones son válidas, las aserciones pasan y todo procede con normalidad. Si no lo son, obtenemos una excepción que nos indica cuáles de nuestras suposiciones son incorrectas. A veces, en lugar de inspeccionar con un depurador o sentencias `print()`, es más rápido soltar un par de aserciones en el código sólo para excluir posibilidades. Veamos un ejemplo:

In [8]:
 # pretender que esto viene de una fuente externa
mylist = [1, 2, 3]

# esto se romperá
assert 4 == len(mylist)

for position in range(4):
    print(mylist[position])

AssertionError: 

En este ejemplo, pretendemos que `mylist` proviene de alguna fuente externa que no controlamos (tal vez la entrada del usuario). El bucle `for` asume que `mylist` tiene cuatro elementos y hemos añadido una aserción para validar esa suposición. El resultado nos dice exactamente dónde está el problema.

Tener en cuenta que al ejecutar un programa con la bandera -O activa hará que Python ignore todas las aserciones. Esto es algo a tener en cuenta si nuestro código depende de aserciones para funcionar.

Las aserciones también permiten un formato más largo que incluye una segunda expresión, como: `assert expression1, expression2`

Típicamente, `expression2` es una cadena que se introduce en la excepción `AssertionError` lanzada por la sentencia. Por ejemplo, si cambiamos la aserción del ejemplo anterior, nos da el siguiente resultado:

In [10]:
 # pretender que esto viene de una fuente externa
mylist = [1, 2, 3]

# esto se romperá
assert 4 == len(mylist), f"Mylist has {len(mylist)} elements"

for position in range(4):
    print(mylist[position])

AssertionError: Mylist has 3 elements

## Donde encontrar información
En la documentación oficial de Python, hay una sección dedicada a la depuración y creación de perfiles, donde puedes leer sobre el framework de depuración `bdb`, y sobre módulos como `faulthandler`, `timeit`, `trace`, `tracemalloc`, y por supuesto `pdb`. Simplemente dirígete a la sección de la biblioteca estándar en la documentación y encontrarás toda esta información muy fácilmente.

# Pautas para la resolución de problemas
En este breve apartado, se mostrarán algunos consejos para la resolución de problemas

## Dónde inspeccionar
Nuestra primera sugerencia se refiere a dónde colocar sus puntos de interrupción de depuración. Independientemente de si estás usando `print()`, una función personalizada, `pdb`, o `logging`, todavía tienes que elegir dónde colocar las llamadas que te proporcionan la información. Algunos lugares son definitivamente mejores que otros, y hay formas de manejar la progresión de depuración que son mejores que otras.

Normalmente evitamos colocar un punto de interrupción dentro de una cláusula `if`. Si la rama que contiene el punto de ruptura no se ejecuta, perdemos la oportunidad de obtener la información que queríamos. A veces puede ser difícil reproducir un error, o el código puede tardar un poco en llegar al punto de interrupción, así que piénsatelo bien antes de colocarlos.

Otra cosa importante es por dónde empezar. Imagina que tienes 100 líneas de código que manejan tus datos. Los datos entran en la línea 1, y de alguna manera están mal en la línea 100. No sabes dónde está el error, así que ¿qué haces? Puedes colocar un punto de interrupción en la línea 1 y recorrer pacientemente las 100 líneas, comprobando los datos en cada paso. En el peor de los casos, 99 líneas (y muchas tazas de café) después, descubres el error. Considere la posibilidad de utilizar un enfoque diferente.

Empiece en la línea 50 e inspeccione. Si los datos son buenos, significa que el fallo se produce más tarde, en cuyo caso coloca tu siguiente punto de interrupción en la línea 75. Si los datos de la línea 50 ya son malos, continúa colocando un punto de interrupción en la línea 25. Luego, repite. A continuación, se repite. Cada vez, avanzas o retrocedes la mitad del salto que diste la última vez.

En el peor de los casos, la depuración iría de 1, 2, 3, ..., 99, de forma lineal, a una serie de saltos como 50, 75, 87, 93, 96, ..., 99, que es mucho más rápido. De hecho, es logarítmica. Esta técnica de búsqueda se llama búsqueda binaria; se basa en un enfoque de divide y vencerás, y es muy eficaz, así que intenta dominarla.

## Usar pruebas para depurar
En la sesión anterior, `Testing`, le presentamos brevemente el desarrollo dirigido por pruebas (TDD). Una práctica de TDD que realmente deberías adoptar, incluso si no te suscribes a TDD en su totalidad, es escribir pruebas que reproduzcan un fallo antes de empezar a cambiar tu código para arreglar el fallo. Hay varias razones para ello. Si tienes un error y todas las pruebas pasan, significa que algo está mal o falta en tu código base de pruebas. Añadir estas pruebas te ayudará a asegurarte de que realmente solucionas el fallo: las pruebas sólo deberían pasar si el fallo ha desaparecido. Por último, tener estas pruebas te protegerá de reintroducir inadvertidamente el mismo fallo de nuevo cuando realices más cambios en tu código.

## Supervisión
La supervisión también es muy importante. Las aplicaciones de software pueden volverse completamente locas y tener contratiempos no deterministas cuando se encuentran con situaciones límite como que la red esté caída, que una cola esté llena o que un componente externo no responda. En estos casos, es importante tener una idea de cuál era el panorama general cuando se produjo el problema y poder correlacionarlo con algo relacionado con él de forma sutil, quizá misteriosa.

Se pueden monitorear los puntos finales de las API, los procesos, la disponibilidad y los tiempos de carga de las páginas web y casi todo lo que se pueda codificar. En general, cuando empiezas una aplicación desde cero, puede ser muy útil diseñarla teniendo en cuenta cómo quieres monitorizarla.

# Creación de perfiles en Python
La creación de perfiles consiste en hacer que la aplicación se ejecute mientras se realiza un seguimiento de varios parámetros diferentes, como el número de veces que se llama a una función y la cantidad de tiempo que se pasa dentro de ella.

La creación de perfiles está estrechamente relacionada con la depuración. Aunque las herramientas y los procesos utilizados son bastante diferentes, ambas actividades implican sondear y analizar el código para entender dónde está la raíz de un problema y luego hacer cambios para solucionarlo. La diferencia es que, en lugar de una salida incorrecta o un bloqueo, el problema que intentamos resolver es un rendimiento deficiente.

A veces el perfil señalará dónde está el cuello de botella del rendimiento, momento en el que tendrás que utilizar las técnicas de depuración que hemos discutido anteriormente en este capítulo para entender por qué una determinada parte del código no funciona tan bien como debería. Por ejemplo, una lógica defectuosa en una consulta a la base de datos puede hacer que se carguen miles de filas de una tabla en lugar de sólo cientos. La creación de perfiles puede mostrarte que una función concreta se llama muchas más veces de lo esperado, momento en el que tendrás que utilizar tus habilidades de depuración para averiguar el motivo y solucionar el problema.

Hay varias formas de perfilar una aplicación Python. Si echas un vistazo a la sección de perfilado en la documentación oficial de la biblioteca estándar, verás que hay dos implementaciones diferentes de la misma interfaz de perfilado, profile y cProfile:
- cProfile está escrito en C y añade comparativamente poca sobrecarga, lo que lo hace adecuado para perfilar programas de larga ejecución.
- profile está implementado en Python puro y, como resultado, añade una sobrecarga significativa a los programas perfilados.

Esta interfaz realiza perfiles deterministas, lo que significa que todas las llamadas a funciones, retornos de funciones y eventos de excepción son monitorizados, y se establecen tiempos precisos para los intervalos entre estos eventos. Otro enfoque, llamado perfilado estadístico, muestrea aleatoriamente la pila de llamadas del programa a intervalos regulares, y deduce dónde se gasta el tiempo.

Este último método suele implicar menos sobrecarga, pero sólo proporciona resultados aproximados. Por otra parte, debido a la forma en que el intérprete de Python ejecuta el código, el perfilado determinista no añade tanta sobrecarga como uno podría pensar, por lo que le mostraremos un ejemplo sencillo utilizando cProfile desde la línea de comandos.

Hay situaciones en las que incluso la sobrecarga relativamente baja de cProfile no es aceptable, por ejemplo, si necesita perfilar código en un servidor web de producción en vivo porque no puede reproducir el problema de rendimiento en su entorno de desarrollo. Para estos casos, realmente necesita un perfilador estadístico. Si estás interesado en perfiles estadísticos para Python, te sugerimos que eches un vistazo a py-spy (https://github.com/benfred/py-spy).

Vamos a calcular triples pitagóricos utilizando el siguiente código:

In [13]:
def calc_triples(mx):
    triples = []
    for a in range(1, mx + 1):
        for b in range(a, mx + 1):
            hypotenuse = calc_hypotenuse(a, b)
            if is_int(hypotenuse):
                triples.append((a, b, int(hypotenuse)))
    return triples

def calc_hypotenuse(a, b):
    return (a**2 + b**2) ** .5

def is_int(n): # n is expected to be a float
    return n.is_integer()

triples = calc_triples(1000)

El script es extremadamente sencillo; iteramos sobre el intervalo [1, mx] con a y b (evitando la repetición de pares estableciendo b >= a) y comprobamos si pertenecen a un triángulo rectángulo. Usamos `calc_hypotenuse()` para obtener la hipotenusa de a y b, y luego, con `is_int()`, comprobamos si es un entero, lo que significa que (a, b, hipotenusa) es un triple pitagórico. Cuando perfilamos este script, obtenemos la información en forma de tabla.

Las columnas son `ncalls` (el número de llamadas a la función), `tottime` (el tiempo total empleado en cada función), `percall` (el tiempo medio empleado en cada función por llamada), `cumtime` (el tiempo acumulado empleado en una función más todas las funciones a las que llama), `percall` (el tiempo medio acumulado empleado por llamada), y `filename:lineno(function)`. Para correr el siguiente código, debes colocar en tu terminal `python -m cProfile ruta_artchivo/nombre_archivo.py`.

Incluso con esta cantidad limitada de datos, todavía podemos inferir información útil sobre este código. En primer lugar, podemos ver que la complejidad temporal del algoritmo que hemos elegido crece con el cuadrado del tamaño de la entrada. El número de llamadas a `calc_hypotenuse()` es exactamente $mx (mx + 1) / 2$. Tres cosas principales suceden dentro de ese bucle: llamamos a `calc_hypotenuse()`, llamamos a `is_int()` y, si se cumple la condición, la añadimos a la lista de triples.

Echando un vistazo a los tiempos acumulados en el informe de perfiles, notamos que el algoritmo ha pasado 0,282 segundos dentro de `calc_hypotenuse()`, que es mucho más que los 0,086 segundos pasados dentro de `is_int()` (tener en cuenta que los tiempos pueden cambiar dependiendo de tu ordenador). Dado que se llamaron el mismo número de veces, veamos si podemos aumentar un poco `calc_hypotenuse()`.

Resulta que sí podemos. Como mencionamos anteriormente, el operador de energía ** es bastante caro, y en `calc_hypotenuse()`, lo estamos usando tres veces. Afortunadamente, podemos transformar fácilmente dos de ellas en multiplicaciones simples, como esta:

In [14]:
def calc_hypotenuse(a, b):
    return (a*a + b*b) ** .5

Este simple cambio debería mejorar las cosas. Si volvemos a ejecutar la generación de perfiles, vemos que 0,282 ahora ha bajado a 0,084. ¡Bien! Esto significa que ahora solo pasamos un 29% del tiempo que antes dentro de `calc_hypotenuse()`.

Veamos si podemos mejorar is_int() también, cambiándolo así:

In [15]:
def is_int(n):
    return n == int(n)

Esta implementación es diferente, y la ventaja es que también funciona cuando n es un número entero. Cuando ejecutamos el perfil contra él, vemos que el tiempo empleado dentro de la función `is_int()` (el cumtime) ha bajado a 0.068 segundos. Curiosamente, el tiempo total empleado en `is_int()` (excluyendo el tiempo empleado en el método `n.is_integer()`), ha aumentado ligeramente, pero en menos tiempo del que solíamos pasar en `n.is_integer()`.

Este ejemplo era trivial, por supuesto, pero suficiente para mostrarle cómo podría perfilar una aplicación. Tener el número de llamadas que se realizan a una función nos ayuda a comprender mejor la complejidad temporal de nuestros algoritmos. Por ejemplo, no creerías cuántos codificadores no ven que esos dos bucles `for` se ejecutan proporcionalmente al cuadrado del tamaño de la entrada.

Una cosa a mencionar: es muy probable que los resultados de la creación de perfiles difieran según el sistema en el que se esté ejecutando. Por lo tanto, es muy importante poder perfilar el software en un sistema que sea lo más parecido posible al sistema en el que se implementa el software, si no es que realmente está en él.

## Cuándo crear un perfil
La creación de perfiles es genial, pero debemos saber cuándo es apropiado hacerlo y qué hacer con los resultados que obtenemos.

Donald Knuth dijo una vez: "La optimización prematura es la raíz de todos los males" y, aunque no lo hubiéramos dicho con tanta fuerza, estamos de acuerdo con él.

Por lo tanto, lo primero y más importante: **la corrección**. Quiere que su código proporcione los resultados correctos, por lo tanto, escriba pruebas, encuentre casos extremos y esfuerce su código de todas las formas que crea que tienen sentido. No sea protector, no guarde cosas en un segundo plano de su cerebro para más adelante porque crea que es poco probable que sucedan. Sea minucioso.

En segundo lugar, **ocúpese de las mejores prácticas de codificación**. Recuerde lo siguiente: legibilidad, extensibilidad, acoplamiento flexible, modularidad y diseño. Aplique los principios de programación orientada a objetos: encapsulación, abstracción, responsabilidad única, abierto/cerrado, etc. Lea sobre estos conceptos. Le abrirán horizontes y ampliarán su forma de pensar sobre el código.

En tercer lugar, **¡refactoriza como una bestia!** La regla de los Boy Scouts dice:

*"Siempre deja el campamento más limpio de lo que lo encontraste"*.

Aplica esta regla a tu código.

Y, finalmente, cuando todo esto se haya solucionado, entonces y solo entonces, ocúpate de optimizar y crear perfiles.

Ejecuta tu generador de perfiles e identifica los cuellos de botella. Cuando tengas una idea de los cuellos de botella que necesitas abordar, comienza primero con el peor. A veces, solucionar un cuello de botella causa un efecto dominó que se expandirá y cambiará la forma en que funciona el resto del código. A veces esto es solo un poco, a veces un poco más, según cómo se diseñó e implementó tu código. Por lo tanto, comienza primero con el problema más grande.

Una de las razones por las que Python es tan popular es que es posible implementarlo de muchas maneras diferentes. Entonces, si tienes problemas para mejorar alguna parte de tu código usando Python puro, nada te impide arremangarte, comprar 200 litros de café y reescribir el lento fragmento de código en C. ¡Garantizado que será divertido!

## Medición del tiempo de ejecución
Antes de terminar esta sesión, toquemos brevemente el tema de la medición del tiempo de ejecución del código. A veces resulta útil medir el rendimiento de pequeñas partes del código para comparar su rendimiento. Por ejemplo, si tiene distintas formas de implementar alguna operación y realmente necesita la versión más rápida, es posible que desee comparar su rendimiento sin crear un perfil de toda la aplicación.

Ya hemos visto algunos ejemplos de medición y comparación de tiempos de ejecución anteriormente en este libro, por ejemplo, en la sesión comprensiones y generadores, cuando comparamos el rendimiento de los bucles for, las comprensiones de listas y la función `map()`. En este punto, nos gustaría presentarle un enfoque mejor, utilizando el módulo `timeit`. Este módulo utiliza técnicas como cronometrar muchas ejecuciones repetidas del código para mejorar la precisión de la medición.

El módulo `timeit` puede ser un poco complicado de usar. Te recomendamos que leas sobre él en la documentación oficial de Python y experimentes con los ejemplos que aparecen allí hasta que entiendas cómo usarlo. Aquí solo daremos una breve demostración del uso de la interfaz de línea de comandos para cronometrar nuestras dos versiones diferentes de `calc_hypotenuse()` del ejemplo anterior:

In [16]:
a = 2 
b = 3
(a ** 2 + b ** 2) ** .5

3.605551275463989

En el código anterior, para poder usar el módulo `timeit` debemos colocar en el terminal `python -m timeit -s 'a=2; b=3' '(a**2 + b**2) ** .5'`.

Así estamos ejecutando el módulo `timeit`, inicializando las variables `a = 2` y `b = 3`, antes de cronometrar la ejecución de `(a ** 2 + b ** 2) ** .5`. En la salida, podemos ver que `timeit` ejecutó 5 repeticiones cronometrando 1000000 iteraciones de bucle ejecutando nuestro cálculo. De esas 5 repeticiones, el mejor tiempo de ejecución promedio en 1000000 iteraciones fue de 254 nanosegundos.

Veamos cómo se ejecuta el cálculo alternativo, `(a * a + b * b) ** .5`:

In [17]:
a = 2 
b = 3
(a * a + b * b) ** .5

3.605551275463989

Tener en cuenta que debemos colocar en el terminal `python -m timeit -s 'a=2; b=3' '(a*a + b*b) ** .5'`.

Esta vez, obtenemos 2000000 de iteraciones de bucle con un promedio de 192 nanosegundos por bucle. Esto confirma una vez más que la segunda versión es significativamente más rápida. La razón por la que obtenemos más iteraciones de bucle en este caso es que `timeit` elige automáticamente la cantidad de iteraciones para garantizar que el tiempo de ejecución total sea de al menos 0,2 segundos. Esto ayuda a mejorar la precisión al reducir el impacto relativo de la sobrecarga de medición.